In [20]:
# Imports
import struct
import traceback
import math
import os
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pandas as pd
import open3d as o3d
from scipy.spatial import Delaunay
from functools import reduce
# %matplotlib qt

In [2]:
# Constants
# Ethernet ----------------------------------------------------------------
SENSOR_FRONT_IP = "192.168.1.10"
SENSOR_RIGHT_IP = "192.168.1.11"
SENSOR_LEFT_IP = "192.168.1.12"
SENSOR_TOP_IP = "192.168.1.13"

# Scans -------------------------------------------------------------------
SCANS_DIRECTORY = r'C:\Users\victo\OneDrive\Desktop\Studies\2024.1\Projeto Integrador\Projeto\DAS5104_Projeto_LiDAR\pointcloud'
SCAN_DIRECTION = "ccw"

# Sensors -----------------------------------------------------------------
SENSOR_TOP_HEIGHT = 2400

# [sensor_x_offset, sensor_y_offset, sensor_z_offset]
SENSOR_RIGHT_TRANSLATION = (1070, 1130, 0)
SENSOR_LEFT_TRANSLATION = (1105, -1130, 0)

# Euler angles [x, y, z]
SENSOR_RIGHT_ROTATION = (0, 0, 0)
SENSOR_LEFT_ROTATION = (0, 0, 0)

BOUNDARIES_PROFILE_X_MIN = 100
BOUNDARIES_PROFILE_X_MAX = 2385
BOUNDARIES_PROFILE_Y_MIN = -1000
BOUNDARIES_PROFILE_Y_MAX = 1000

BOUNDARIES_ZAXIS_X_MIN = -2300
BOUNDARIES_ZAXIS_X_MAX = -400
BOUNDARIES_ZAXIS_Y_MIN = -7000
BOUNDARIES_ZAXIS_Y_MAX = -100

# Output -----------------------------------------------------------------
OUTPUT_DIRECTORY = r'outputs'

In [3]:
def ntp64_to_seconds(integer):
    # Upper 32 bits for seconds
    seconds = integer >> 32

    # Lower 32 bits for fractional seconds
    fractional_seconds = integer & 0xFFFFFFFF
    fractional_seconds = fractional_seconds / 0x100000000

    return round(seconds + fractional_seconds, 3)

In [4]:
def polar_to_xy(distances: list, first_angle: int, angular_increment: int):
    first_angle /= 10000
    angular_increment /= 10000

    xy = list()

    for i, distance in enumerate(distances):
        # Invalid measurements return 0xFFFFFFFF
        if distance == 4_294_967_295:
            continue

        angle = (first_angle + i * angular_increment) * math.pi / 180.0

        x = round(distance * math.cos(angle))
        y = round(distance * math.sin(angle))

        xy.append((x, y))

    return xy

In [5]:
def process_binary_file(file_path: str):
    file = open(file_path, "rb")
    data = file.read()
    file.close()

    scans = dict()
    magic_byte = struct.pack("H", 0xa25c)

    for packet in data.split(magic_byte):

        if len(packet) <= 10:
            continue

        try:
            packet_size = struct.unpack("I", packet[2:6])[0] - len(magic_byte)
            header_size = struct.unpack("H", packet[6:8])[0] - len(magic_byte)
            scan_number = struct.unpack("H", packet[8:10])[0]
            timestamp_raw = struct.unpack("Q", packet[12:20])[0]
            first_angle = struct.unpack("i", packet[42:46])[0]
            angular_increment = struct.unpack("i", packet[46:50])[0]
        except Exception as e:
            print(f"Package from IP {os.path.basename(file_path).split('.bin')[0]} corrupted...")
            print(e)
            continue

        if len(packet) != packet_size:
            print(f"Package from IP {os.path.basename(file_path).split('.bin')[0]} corrupted")
            print('Expected packet size:', packet_size)
            print('Received packet size:', len(packet))
            continue

        if scan_number not in scans:
            scans[scan_number] = dict()
            scans[scan_number]["xy"] = list()
            scans[scan_number]["timestamp"] = ntp64_to_seconds(timestamp_raw)

        payload = packet[header_size:]  # list[uint32] - 4byte
        distances = struct.unpack(f"{len(payload) // 4}I", payload[:len(payload) // 4 * 4])

        scans[scan_number]["xy"].extend(polar_to_xy(distances, first_angle, angular_increment))

    return scans

In [6]:
def calculate_z_axis(scans_front: dict, x_min: int, x_max: int, y_min: int, y_max: int):
    z_axis = {}
    xyz_front = []

    for i, scan_key in enumerate(sorted(scans_front.keys())):
        z_axis[i] = y_min

        for xy in scans_front[scan_key]["xy"]:
            x = xy[0]
            y = xy[1]
            z = i * 5

            if x <= x_min or x >= x_max or y <= y_min or y >= y_max:
                continue

            if y > z_axis[i]:
                z_axis[i] = y

            xyz_front.append((x, y, z))

    return z_axis, xyz_front

In [7]:
def reconstruct_z_axis(scans: dict, z_axis: dict) -> list[tuple[int, int, int]]:
    xyz = list()

    for key in zip(sorted(scans.keys()), sorted(z_axis.keys())):
        for xy in scans[key[0]]["xy"]:
            x = xy[0]
            y = xy[1]
            z = z_axis[key[1]]

            xyz.append((x, y, z))

    return xyz

In [8]:
def transform(points, rotation: tuple[int, int, int], translation: tuple[int, int, int]):
    try:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
    
        if rotation != (0, 0, 0):
            rotation_matrix = pcd.get_rotation_matrix_from_xyz(rotation)
            pcd.rotate(rotation_matrix, center=(0, 0, 0))

        if translation != (0, 0, 0):
            pcd.translate(translation)
    except Exception as e:
        print(f'Error in point {points}: {traceback.format_exc()}')

    return np.asarray(pcd.points)

In [9]:
def remove_boundaries(points, x_min: int, x_max: int, y_min: int, y_max: int):
  return [p for p in points if not (p[0] <= x_min or p[0] >= x_max or p[1] <= y_min or p[1] >= y_max)]

In [10]:
def plot_point_cloud(arr, title):
  df=pd.DataFrame(data=arr, columns=['x', 'y', 'z'])
  fig = px.scatter_3d(df, x='x', y='y', z='z', color='y', title=title, color_continuous_scale=px.colors.sequential.Cividis_r)
  fig.update_traces(marker_size=2)
  fig.show()

In [11]:
def filter_point_cloud(points, voxel_size, nb_neighbors, std_ratio, nb_points, radius):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(points)

  xyz_1 = pcd.voxel_down_sample(voxel_size=voxel_size)
  xyz_2, _ = xyz_1.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
  xyz_3, _ = xyz_2.remove_radius_outlier(nb_points=nb_points, radius=radius)

  return np.asarray(xyz_3.points)

In [12]:
def get_point_cloud(obj):
  scans_front = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_FRONT_IP}.bin'))
  scans_right = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_RIGHT_IP}.bin'))
  scans_left = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_LEFT_IP}.bin'))
  scans_top = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_TOP_IP}.bin'))
  
  z_axis, _ = calculate_z_axis(
    scans_front,
    BOUNDARIES_ZAXIS_X_MIN,
    BOUNDARIES_ZAXIS_X_MAX,
    BOUNDARIES_ZAXIS_Y_MIN,
    BOUNDARIES_ZAXIS_Y_MAX,
  )
  
  xyz_right = reconstruct_z_axis(scans_right, z_axis)
  # plot_point_cloud(xyz_right, 'Scans do sensor right com reconstrução do eixo Z')
  xyz_left = reconstruct_z_axis(scans_left, z_axis)
  xyz_top = reconstruct_z_axis(scans_top, z_axis)
  
  xyz_right = transform(xyz_right, SENSOR_RIGHT_ROTATION, SENSOR_RIGHT_TRANSLATION)
  # plot_point_cloud(xyz_right, 'Scans do sensor right após translação')
  xyz_left = transform(xyz_left, SENSOR_LEFT_ROTATION, SENSOR_LEFT_TRANSLATION)
  
  xyz_right = remove_boundaries(
    xyz_right,
    BOUNDARIES_PROFILE_X_MIN,
    BOUNDARIES_PROFILE_X_MAX,
    BOUNDARIES_PROFILE_Y_MIN,
    BOUNDARIES_PROFILE_Y_MAX,
  )
  
  # plot_point_cloud(xyz_right, 'Scans do sensor right sem dados fora dos limites')

  xyz_left = remove_boundaries(
    xyz_left,
    BOUNDARIES_PROFILE_X_MIN,
    BOUNDARIES_PROFILE_X_MAX,
    BOUNDARIES_PROFILE_Y_MIN,
    BOUNDARIES_PROFILE_Y_MAX,
  )

  xyz_top = remove_boundaries(
    xyz_top,
    BOUNDARIES_PROFILE_X_MIN,
    BOUNDARIES_PROFILE_X_MAX,
    BOUNDARIES_PROFILE_Y_MIN,
    BOUNDARIES_PROFILE_Y_MAX,
  )

  xyz_right = filter_point_cloud(xyz_right, 15, 40, 0.1, 25, 50)
  # plot_point_cloud(xyz_right, 'Scans do sensor right com filtros')
  xyz_left = filter_point_cloud(xyz_left, 15, 40, 0.1, 25, 50)
  xyz_top = filter_point_cloud(xyz_top, 15, 60, 0.06, 25, 120)
  
  xyz = list()
  xyz.extend(xyz_right)
  xyz.extend(xyz_left)
  xyz.extend(xyz_top)

  xyz = transform(xyz, (0, 0, -math.pi/2), (0, SENSOR_TOP_HEIGHT, 0))
  plot_point_cloud(xyz, 'Point cloud reconstruída')

  return xyz

In [13]:
def convex_hull(xyz):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(xyz)
  pcd.estimate_normals()

  hull, _ = pcd.compute_convex_hull()
  hull.orient_triangles()

  hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
  hull_ls.paint_uniform_color((1, 0, 0))

  volume = hull.get_volume()

  # o3d.visualization.draw_geometries([pcd, hull])

  return volume

In [14]:
def alpha_shapes(xyz):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(xyz)
  pcd.estimate_normals()

  alpha = 25

  mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
  mesh.orient_triangles()

  volume = mesh.get_volume()
  
  # o3d.visualization.draw_geometries([pcd, mesh])

  return volume

In [15]:
def _volume_under_triangle(triangle):
    p1, p2, p3 = triangle
    x1, z1, y1 = p1
    x2, z2, y2 = p2
    x3, z3, y3 = p3

    return abs((z1+z2+z3)*(x1*y2-x2*y1+x2*y3-x3*y2+x3*y1-x1*y3)/6)

def _get_triangles_vertices(triangles, vertices):
    triangles_vertices = []

    for triangle in triangles:
        new_triangles_vertices = [vertices[triangle[0]], vertices[triangle[1]], vertices[triangle[2]]]
        triangles_vertices.append(new_triangles_vertices)

    return np.array(triangles_vertices)

def delaunay(xyz):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    xyz = np.asarray(pcd.points)
    xy_catalog = []

    for point in xyz:
        xy_catalog.append([point[0], point[2]])

    tri = Delaunay(np.array(xy_catalog))

    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(xyz)
    mesh.triangles = o3d.utility.Vector3iVector(tri.simplices)
    
    # o3d.visualization.draw_geometries([pcd, mesh])

    volume = reduce(lambda a, b:  a + _volume_under_triangle(b),
                    _get_triangles_vertices(mesh.triangles, mesh.vertices), 0)

    return volume

In [16]:
def ball_pivoting(xyz):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(xyz)
  pcd.estimate_normals()

  radii = [50, 25, 30]
  mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))

  volume = mesh.get_volume()
  return volume


In [17]:
def estimate_volume(xyz):
  try:
    print(f'Convex hull: {convex_hull(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método do convex hull:', e)
  try:
    print(f'Alpha shapes: {alpha_shapes(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método das alpha shapes:', e)
  try:
    print(f'Ball pivoting: {ball_pivoting(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método de ball pivoting:', e)
  try:
    print(f'Triangulação de Delaunay: {delaunay(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método da triangulação de delaunay:', e)

In [19]:
obj = 'caixa_grande'

point_cloud = get_point_cloud(obj)
estimate_volume(point_cloud)

Package from IP 192.168.1.12 corrupted...
unpack requires a buffer of 8 bytes
Package from IP 192.168.1.12 corrupted
Expected packet size: -2
Received packet size: 1130


Convex hull: 292455880.9166667 mm³
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
O volume não pode ser estimado através do método das alpha shapes: [Open3D Error] (double __cdecl open3d::geometry::TriangleMesh::GetVolume(void) const) D:\a\Open3D\Open3D\cpp\open3d\geometry\TriangleMesh.cpp:1201: The mesh is not watertight, and the volume cannot be computed.

O volume não pode ser estimado através do método de ball pivoting: [Open3D Error] (double __cdecl open3d::geometry::TriangleMesh::GetVolume(void) const) D:\a\Open3D\Open3D\cpp\open3d\geometry\TriangleMesh.cpp:1201: The mesh is not watertight, and the volume cannot be computed.

Triangulação de Delaunay: 323441250.5104165 mm³
